In [8]:
# pip install plotly
# pour visualization de BERTopic

# !pip install bertopic

# pip install SentenceTransformer

In [14]:
import pandas as pd
import spacy
import torch


df = pd.read_csv("contrib_from_csv.csv", sep = ",", dtype= str)

# sample d'un très petit échantillon pour cet ordi
df = df.sample(10000)

In [15]:
join_breaklines = lambda x : x.replace("\x0b"," ") 
join_breaklinesn = lambda x : x.replace("\n"," ") 
kill_texte_illisible = lambda x : x.replace("texte illisible", "")
kill_illisible = lambda x : x.replace("illisible", "")
kill_illisible_plural = lambda x : x.replace("illisibles", "")


In [16]:
df["dupyo"] = df["Contribution"].apply(join_breaklines)

In [17]:
df["clean"] = df["dupyo"].apply(join_breaklinesn)
df["clean"] = df["clean"].apply(kill_texte_illisible)
df["clean"] = df["clean"].apply(kill_illisible)
df["clean"] = df["clean"].apply(kill_illisible_plural)

In [18]:
pd.set_option('display.max_colwidth', None)
df["clean"].head

<bound method NDFrame.head of 23271                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [20]:
nlp = spacy.load("fr_core_news_md", disable = ["parser", "ner"])

# Process large text as a stream via `nlp.pipe()` and iterate over the results, extracting lemmas
lemma_text_list = []
for doc in nlp.pipe(df["clean"]):
    lemma_text_list.append(" ".join(token.lemma_ for token in doc))
df["text_lemma"] = lemma_text_list

# 4mn 58s pour 10000

KeyboardInterrupt: 

In [27]:
documents = df.clean.tolist()
documents[0]

"Le 08/01/2019 Inscrire dans la constitution : - La loi sur l'avortement - Le mariage homosexuel - L'interdiction de la peine de mort Relevé le taux du SMIC à 1800 Euros - Les minima sociaux - Augmentation de scolaire - Indexation des Retraits à la Hausse. - Relevé les pensions des Handicaps - Rétablir ISF - Arrêt Imédiat de l'évasion fiscale - Taxation des revenus financier - Accueil décent des réfugies Josy GADOZI retraiter louis PUAZZOLLE"

In [28]:
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from spacy.lang.en.stop_words import STOP_WORDS as en_stop

final_stopwords_list = list(fr_stop) + list(en_stop) + ["\n"]
final_stopwords_set = set(final_stopwords_list)
def remove_stopwords(text):
    text = [word for word in text if word not in final_stopwords_set]
    return text

In [29]:
nostopword = remove_stopwords(documents)

In [30]:
def lemmatize(docs, allowed_postags = ["NOUN", "ADJ", "VERB", "ADV"]):
  '''
  Performs lemmization of input documents.
  Args:
    - docs: list of strings with input documents
    - allowed_postags: list of accepted Part of Speech (POS) types
  Output:
    - list of strings with lemmatized input
  '''
  nlp = spacy.load("fr_core_news_md", disable = ["parser", "ner"])
  nlp.max_length = 3000000 #this is possible as we're not using parser or ner.
  lemmatized_docs = []
  for doc in docs:
    doc = nlp(doc)
    tokens = []
    for token in doc:
      if token.pos_ in allowed_postags:
        tokens.append(token.lemma_)
    lemmatized_docs.append(" ".join(tokens))
  return (lemmatized_docs)

In [31]:
lemmatized = lemmatize(nostopword)

In [13]:
# import umap.umap_ as mp
# UMAP = mp.UMAP()

In [12]:
from sentence_transformers import SentenceTransformer

In [24]:
# import umap.umap_ as UMAP
# from umap-learn import UMAP

# import umap.umap_ as umap 


In [1]:
import pandas as pd 
import numpy as np
import re
from bertopic import BERTopic

In [32]:
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(lemmatized, show_progress_bar=False)

In [33]:
topic_model = BERTopic(verbose=True)
topics, probs = topic_model.fit_transform(lemmatized, embeddings)

2023-04-12 01:42:16,126 - BERTopic - Reduced dimensionality
2023-04-12 01:42:16,185 - BERTopic - Clustered reduced embeddings


In [34]:
# Run the visualization with the original embeddings
topic_model.visualize_documents(lemmatized, embeddings=embeddings)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
# reduced_embeddings = reducer(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
# topic_model.visualize_documents(lemmatized, embeddings)

In [35]:
topic_model.get_topics()

{-1: [('ne', 0.022731714354859746),
  ('plus', 0.021453778632825438),
  ('pas', 0.019894913339585),
  ('tout', 0.01898614963180232),
  ('pouvoir', 0.017202595957610464),
  ('devoir', 0.01674957541317747),
  ('avoir', 0.01586372798614495),
  ('faire', 0.014745071779880216),
  ('public', 0.012490371585975288),
  ('service', 0.01145194706561949)],
 0: [('retraite', 0.045389284387298376),
  ('csg', 0.02537247037098494),
  ('ne', 0.025206032613044996),
  ('tout', 0.024557326487290625),
  ('taxe', 0.024050118759273393),
  ('pas', 0.023949639416527498),
  ('plus', 0.022773990293179277),
  ('augmentation', 0.022643780013090996),
  ('suppression', 0.02179929693877667),
  ('retrait', 0.021474655519559287)],
 1: [('vote', 0.038862364358779586),
  ('lection', 0.034855232126372634),
  ('falloir', 0.030757202854272858),
  ('blanc', 0.025036114795945606),
  ('citoyen', 0.024994159583755702),
  ('rfrendum', 0.024990543788720003),
  ('tout', 0.02418115169104518),
  ('oui', 0.023921075089503373),
  ('no

In [36]:
fig = topic_model.visualize_barchart()
fig.show()

In [37]:
topic_model.get_topic_freq().head(11)

,Topic,Count
0,-1,535
1,0,77
2,1,51
3,2,38
4,3,37
5,4,29
6,5,27
7,6,23
8,7,23
9,8,21


In [44]:
topic_model.get_topic(1)
topic_model.get_topic(2)


[('ne', 0.026901585374861776),
 ('pas', 0.0254711965674036),
 ('prsident', 0.022553018443582578),
 ('tout', 0.01970157356289218),
 ('faire', 0.01858790700848864),
 ('franais', 0.015585029883288033),
 ('pouvoir', 0.01545914665941066),
 ('plus', 0.01459039804516184),
 ('avoir', 0.01443628561767832),
 ('pays', 0.014242507513567843)]

In [45]:
#  les echantillons sont trop petits pour avoir des topics visualisés
fig = topic_model.visualize_topics()
fig.show()

In [47]:
# topic_model.visualize_documents(lemmatized)

In [48]:
topic_model.visualize_hierarchy()

: 